In [2]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import main as m
import regex as re

In [7]:
def get_slot_url(scrape_url):

    html = requests.get(scrape_url)

    soup = BeautifulSoup(html.content, 'html.parser')

    # locate urls on page
    results = soup.find(id='list')

    results = results.find_all('a', class_='card-tip')

    # convert urls to string and make a list
    urls = [str(result['href']) for result in results]

    return urls

In [8]:
one_hand = get_slot_url('https://arkhamdb.com/find?q=z%3Ahand&decks=player')

In [9]:
two_hand = get_slot_url('https://arkhamdb.com/find?q=z%3A%22hand+x2%22&decks=player')

In [10]:
accessory = get_slot_url('https://arkhamdb.com/find?q=z%3Aaccessory&decks=player')

In [11]:
ally = get_slot_url('https://arkhamdb.com/find?q=z%3Aally&decks=player')

In [12]:
one_arcane = get_slot_url('https://arkhamdb.com/find?q=z%3Aarcane&decks=player')

In [13]:
two_arcane = ['https://arkhamdb.com/card/06328']

In [14]:
body = get_slot_url('https://arkhamdb.com/find?q=z%3Abody&decks=player')

In [15]:
tarot = get_slot_url('https://arkhamdb.com/find?q=z%3Atarot&sort=name&view=list&decks=player&spoilers=hide')

In [18]:
if url in one_hand:
    
    .append('One Handed')
    
elif url in two_hand:
    
    .append('Two Handed')
    
elif url in accessory:

    .append('Accessory')
    
elif url in ally:
    
    .append('Ally')
    
elif url in one_arcane:
    
    .append('One Arcane')
    
elif url in two_arcane:
    
    .append('Two Arcane')
    
    

In [ ]:
one_hand = get_slot_url('https://arkhamdb.com/find?q=z%3Ahand&decks=player')

two_hand = get_slot_url('https://arkhamdb.com/find?q=z%3A%22hand+x2%22&decks=player')

accessory = get_slot_url('https://arkhamdb.com/find?q=z%3Aaccessory&decks=player')

ally = get_slot_url('https://arkhamdb.com/find?q=z%3Aally&decks=player')

one_arcane = get_slot_url('https://arkhamdb.com/find?q=z%3Aarcane&decks=player')

two_arcane = ['https://arkhamdb.com/card/06328']

df = pd.read_csv("investigators.csv")

In [31]:
def get_slot(url):
    
    one_hand = get_slot_url('https://arkhamdb.com/find?q=z%3Ahand&decks=player')

    two_hand = get_slot_url('https://arkhamdb.com/find?q=z%3A%22hand+x2%22&decks=player')
    
    if url in one_hand:
        
        slot = "One Handed"
        
    if url in two_hand:
        
        slot = "Two Handed"
        
    else:
        
        slot = "None"

In [32]:
df['slot'] = df.url.apply(get_slot)

KeyboardInterrupt: 

In [ ]:
df.head()

In [3]:
catagory = "asset"

full_results = []

if catagory in ('investigator', 'skill'):
    
    pages = 1
    
elif catagory == 'event':
    
    pages = 2
    
elif catagory == 'asset':
    
    pages = 4

for page in range(1,pages+1):

    url = f'https://arkhamdb.com/find?q=t%3A{catagory}&view=list&sort=name&decks=player&page={page}'

    # create request and soup objects
    html = requests.get(url)

    soup = BeautifulSoup(html.content, 'html.parser')

    # locate urls on page
    results = soup.find(id='list')

    results = results.find_all('a', class_='card-tip')

    # convert urls to string and make a list
    results = [str(result['href']) for result in results]
               
    full_results.extend(results)

In [57]:
##########################################Helper functions#########################################################
def get_soup(url):
    '''
       Takes in a url for a card
       Returns html request result parsed using beautiful soup'''

    # create request and soup objects
    html = requests.get(url)

    soup = BeautifulSoup(html.content)

    # locate urls on page and return
    return soup

def get_cost_xp(results):
    
    cost_xp = str(results.find('div', class_='card-props'))

    try:
    
        cost = re.search(r'Cost:\s+(\d+)', cost_xp).group(1)
        
    except:
        
        cost = 0

    try:
    
        xp = re.search(r'XP:\s+(\d+)', cost_xp).group(1)
        
    except:
        
        xp = 0
    
    return cost, xp


def get_subtitle(results):
    
    try:
        
        sub_title = results.find('div', class_='card-subname small').text.replace('\n', '').replace('\t', '')
        
    except:
        
        sub_title = ''
        
    return sub_title



def get_traits(results):
    
    try:
        
        traits = results.find('p', class_='card-traits').text.replace('\n', '').replace('\t', '')
        
    except:
        
        traits = ''
        
    return traits


def get_icons(results):
    '''Child of get_card_traits
       Takes in request results for an arkhamdb page containing player card data
       Returns a string represintation of skill test icons on the card'''
      
    icons = ''

    # list containing each icon type
    icon_types = ['wild', 'willpower', 'combat', 'agility', 'intellect']

    # itterate through icon types
    for stat in icon_types:

        # get number of that icon on card from request results
        num_icons = len(results.find_all('span', class_=f'icon icon-{stat} color-{stat}'))

        # add that icon name to a string for each time it appears in request results
        for icon in range(num_icons):

            icons += f'{stat} '
            
    return icons.upper()[:-1]


def get_ability(results, faction):
    '''Child of get_card_traits
        Takes in request results for an arkhamdb page containing player card data
        Returns a string represintation of skill test icons on the card'''
    
    # gets html object and convert to string
    ability_string = str(results.find_all('div', class_=f'card-text border-{faction.lower()}'))
    
    # convert html to string, replace icons in text with string represintations
    ability_text = get_text_for_icon(ability_string)
    
    ability_text = clean_html(ability_text)
    
    return ability_text



def get_text_for_icon(text):

    # replace icon html with matching word in all caps
    icon_types = [
                  'action',
                  'reaction',
                  'wild', 
                  'willpower', 
                  'combat', 
                  'agility', 
                  'intellect', 
                  'wild',
                  'curse', 
                  'bless',
                  'rogue',
                  'survivor',
                  'seeker',
                  'guardian',
                  'mystic',
                  'neutral',
                  'skull',
                  'tablet',
                  'cultist',
                  'elder sign']
    
    for icon in icon_types:
    
        text = text.replace(f'<span class="icon-{icon}" title="{icon.capitalize()}"></span>', 
                                  f'{icon.upper()}')
        
        text = text.replace(f'<div class="card-text border-{icon}">\n<p>', '')

    text = text.replace(f'<span class="icon-wild" title="Any Skill"></span>', 'WILD')

    text = text.replace(f'<span class="icon-elder_sign" title="Elder Sign"></span>', 'ELDER_SIGN')

    text = text.replace(f'<span class="icon-elder_sign" title="Elder Thing"></span>', 'ELDER_THING')

    text = text.replace(f'<span class="icon-lightning" title="Fast Action"></span>', 'FAST ACTION')

    text = text.replace(f'<span class="icon-auto_fail" title="Auto Fail"></span>', 'TENTACLES')

    return text


def clean_html(text):

    # delete extraneous html
    dirt = [
            '</p>\n</div>',
            '</p>',
            '<p>',
            '<b>',
            '</b>',
            '<br/',
            '<1>',
            '<i>',
            '</i>',
            '><span>']
    
    for item in dirt:
        
        text = text.replace(item,'')

    return text


def get_stam_line(results):
    ''' scrapes website for investigator health and sanity'''

    stat_line = results.find('div', class_='card-props').find_next().text.replace('\n', '').replace('\t', '')

    stat_list = [int(char) for char in str(stat_line) if char.isnumeric() == True]

    return stat_list[0], stat_list[1]


def get_asset_stam_line(results):
    
    results = results.find('div').text.replace('\n', '').replace('\t', '') 
    
    health = re.search(r'Health:\s+(\d+)', stam_line).group(1)
    
    sanity = re.search(r'Sanity:\s+(\d+)', stam_line).group(1)
    
    return health, sanity

In [58]:
url = 'https://arkhamdb.com/card/01563'

results = get_soup(url)

In [59]:
def get_asset_traits():
    
    title = results.find('a', class_='card-name card-tip').text.replace('\n', '').replace('\t', '')

    sub_title = get_subtitle(results)

    faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

    tipe = results.find('span', class_='card-type').text.replace('\n', '').replace('\t', '')

    traits = get_traits(results)

    cost, xp = get_cost_xp(results)

    icons = get_icons(results)

    ability = get_ability(results, faction)

    health, sanity = get_asset_stam_line(results)

    artist = results.find('div', class_='card-illustrator').text.replace('\n', '').replace('\t', '')

    expansion = results.find('div', class_='card-pack').text.replace('\n', '').replace('\t', '').replace('.', '')
    
    flavor = results.find('div', class_='card-flavor small').text.replace('\n', '').replace('\t', '')

    

In [ ]:
health

In [ ]:
sanity

In [44]:
re.search(r'Health:\s+(\d+)', stam_line).group(1)

'1'

In [45]:
re.search(r'Sanity:\s+(\d+)', stam_line).group(1)

'2'

In [18]:
title

'Armor of Ardennes'

In [19]:
sub_title

''

In [20]:
faction

'Guardian'

In [21]:
tipe

'Asset. '

In [22]:
traits

'Item. Armor. Relic.'

In [23]:
cost

'4'

In [25]:
xp

'5'

In [26]:
icons

'WILLPOWER WILLPOWER COMBAT COMBAT'

In [27]:
ability

'[REACTION When damage is assigned to Armor of Ardennes, exhaust Armor of Ardennes: Cancel 1 of that damage.]'

In [28]:
artist

'Adam Lane'

In [29]:
flavor

''

In [30]:
expansion

'Dim Carcosa #305'

In [11]:

num = 0

full_results = []

while True:    
    
    num += 1
    
    url = f'https://arkhamdb.com/find?q=t%3Aevent&view=list&sort=name&decks=player&page={num}'
    
    print(url)
    
    try:
        
        # create request and soup objects
        html = requests.get(url)
        print("found Page", num)
    except:
        
        print("break page", num)
        break

    soup = BeautifulSoup(html.content, 'html.parser')

    # locate urls on page
    results = soup.find(id='list')

    results = results.find_all('a', class_='card-tip')

    # convert urls to string and make a list
    results = [str(result['href']) for result in results]
    
    full_results.extend(results)
    
full_results

https://arkhamdb.com/find?q=t%3Aevent&view=list&sort=name&decks=player&page=1
found Page 1
https://arkhamdb.com/find?q=t%3Aevent&view=list&sort=name&decks=player&page=2
found Page 2
https://arkhamdb.com/find?q=t%3Aevent&view=list&sort=name&decks=player&page=3
found Page 3
https://arkhamdb.com/find?q=t%3Aevent&view=list&sort=name&decks=player&page=4
found Page 4
https://arkhamdb.com/find?q=t%3Aevent&view=list&sort=name&decks=player&page=5
found Page 5
https://arkhamdb.com/find?q=t%3Aevent&view=list&sort=name&decks=player&page=6
found Page 6
https://arkhamdb.com/find?q=t%3Aevent&view=list&sort=name&decks=player&page=7
found Page 7
https://arkhamdb.com/find?q=t%3Aevent&view=list&sort=name&decks=player&page=8
found Page 8
https://arkhamdb.com/find?q=t%3Aevent&view=list&sort=name&decks=player&page=9
found Page 9
https://arkhamdb.com/find?q=t%3Aevent&view=list&sort=name&decks=player&page=10
found Page 10
https://arkhamdb.com/find?q=t%3Aevent&view=list&sort=name&decks=player&page=11
found Pag

['https://arkhamdb.com/card/03304',
 'https://arkhamdb.com/card/52002',
 'https://arkhamdb.com/card/06156',
 'https://arkhamdb.com/card/08021',
 'https://arkhamdb.com/card/60114',
 'https://arkhamdb.com/card/60123',
 'https://arkhamdb.com/card/08112',
 'https://arkhamdb.com/card/10072',
 'https://arkhamdb.com/card/03189',
 'https://arkhamdb.com/card/09067',
 'https://arkhamdb.com/card/05016',
 'https://arkhamdb.com/card/02151',
 'https://arkhamdb.com/card/02107',
 'https://arkhamdb.com/card/60225',
 'https://arkhamdb.com/card/01684',
 'https://arkhamdb.com/card/02261',
 'https://arkhamdb.com/card/05315',
 'https://arkhamdb.com/card/02225',
 'https://arkhamdb.com/card/06160',
 'https://arkhamdb.com/card/03022',
 'https://arkhamdb.com/card/01079',
 'https://arkhamdb.com/card/01579',
 'https://arkhamdb.com/card/60517',
 'https://arkhamdb.com/card/60524',
 'https://arkhamdb.com/card/10049',
 'https://arkhamdb.com/card/04028',
 'https://arkhamdb.com/card/05319',
 'https://arkhamdb.com/card/

In [ ]:
# Pages of results

invest = 1
skills = 1
assets = 4
events = 2


In [40]:
df = pd.read_html('https://arkhamdb.com/find?q=z%3Abody&decks=player')[0]
df = df[["Name"]]

In [41]:
df.Name = df.Name.apply(lambda value: re.search( r'([A-Za-z\s.]+)', str(value)).group(0).strip())

In [43]:
df.Name.to_list()

['Armor of Ardennes',
 'Astral Mirror',
 'Backpack',
 'Backpack',
 'Bandolier',
 'Bandolier',
 'Bulletproof Vest',
 'Bulletproof Vest',
 'Fine Clothes',
 'Heavy Furs',
 'Hunter',
 'Lab Coat',
 'Leather Coat',
 'Leather Coat',
 'Leather Coat',
 'Leather Jacket',
 'Living Ink',
 'Protective Gear',
 'Quickdraw Holster',
 'Robes of Endless Night',
 'Robes of Endless Night',
 'Stylish Coat',
 'Tool Belt',
 'Trench Coat']

In [11]:
y = pd.read_html('https://arkhamdb.com/card/01514')

ValueError: No tables found

In [2]:
import selenium

In [3]:
# investigator 
# ablilities with embedid values not capturing correctly
# Complex deck reqs not being captured correctly
# some must includes not being captured correctly

In [4]:
invest, assets, events, skills = m.get_urls_by_type()


Getting investigator URL's...
Getting asset URL's...
Getting event URL's...
Getting skill URL's...


In [5]:
##########################################Helper functions#########################################################
def get_soup(url):
    '''
       Takes in a url for a card
       Returns html request result parsed using beautiful soup'''

    # create request and soup objects
    html = requests.get(url)

    soup = BeautifulSoup(html.content)

    # locate urls on page and return
    return soup


def get_text_for_icon(text):

    # replace icon html with matching word in all caps
    icon_types = [
                  'action',
                  'reaction',
                  'wild', 
                  'willpower', 
                  'combat', 
                  'agility', 
                  'intellect', 
                  'wild',
                  'curse', 
                  'bless',
                  'rogue',
                  'survivor',
                  'seeker',
                  'guardian',
                  'mystic',
                  'neutral',
                  'skull',
                  'tablet',
                  'cultist',
                  'elder sign']
    
    for icon in icon_types:
    
        text = text.replace(f'<span class="icon-{icon}" title="{icon.capitalize()}"></span>', 
                                  f'{icon.upper()}')
        
        text = text.replace(f'<div class="card-text border-{icon}">\n<p>', '')

    text = text.replace(f'<span class="icon-wild" title="Any Skill"></span>', 'WILD')

    text = text.replace(f'<span class="icon-elder_sign" title="Elder Sign"></span>', 'ELDER_SIGN')

    text = text.replace(f'<span class="icon-elder_sign" title="Elder Thing"></span>', 'ELDER_THING')

    text = text.replace(f'<span class="icon-lightning" title="Fast Action"></span>', 'FAST ACTION')

    text = text.replace(f'<span class="icon-auto_fail" title="Auto Fail"></span>', 'TENTACLES')

    return text

def clean_html(text):

    # delete extraneous html
    dirt = [
            '</p>\n</div>',
            '</p>',
            '<p>',
            '<b>',
            '</b>',
            '<br/',
            '<1>',
            '<i>',
            '</i>',
            '><span>']
    
    for item in dirt:
        
        text = text.replace(item,'')

    return text

In [23]:
def get_xp(results):
    ''' Child of get_card_traits
        Takes in an BS object containing card information
        Returns XP cost of card if it exists
        Otherwise returns 0'''
    
    # locate xp info in object
    xp = results.find('div', class_='card-props')
    
    # if there is no value return 0
    if xp == None:
        
        return 0
    
    # otherwise return perform minor cleaning on value and return
    else:
        
        return re.search('[0-9]', str(xp)).group()
    
    
def get_icons(results):
    '''Child of get_card_traits
       Takes in request results for an arkhamdb page containing player card data
       Returns a string represintation of skill test icons on the card'''
      
    icons = ''

    # list containing each icon type
    icon_types = ['wild', 'willpower', 'combat', 'agility', 'intellect']

    # itterate through icon types
    for stat in icon_types:

        # get number of that icon on card from request results
        num_icons = len(results.find_all('span', class_=f'icon icon-{stat} color-{stat}'))

        # add that icon name to a string for each time it appears in request results
        for icon in range(num_icons):

            icons += f'{stat} '
            
    return icons.upper()[:-1]


#####find changes to find_all#######################################################################
def get_ability(results, faction):
    '''Child of get_card_traits
        Takes in request results for an arkhamdb page containing player card data
        Returns a string represintation of skill test icons on the card'''
    
    # gets html object and convert to string
    ability_string = str(results.find_all('div', class_=f'card-text border-{faction.lower()}'))
    
    # convert html to string, replace icons in text with string represintations
    ability_text = get_text_for_icon(ability_string)
    
    ability_text = clean_html(ability_text)
    
    return ability_text


def get_text_for_icon(text):

    # replace icon html with matching word in all caps
    icon_types = [
                  'action',
                  'reaction',
                  'wild', 
                  'willpower', 
                  'combat', 
                  'agility', 
                  'intellect', 
                  'wild',
                  'curse', 
                  'bless',
                  'rogue',
                  'survivor',
                  'seeker',
                  'guardian',
                  'mystic',
                  'neutral',
                  'skull',
                  'tablet',
                  'cultist',
                  'elder sign']
    
    for icon in icon_types:
    
        text = text.replace(f'<span class="icon-{icon}" title="{icon.capitalize()}"></span>', 
                                  f'{icon.upper()}')
        
        text = text.replace(f'<div class="card-text border-{icon}">\n<p>', '')

    text = text.replace(f'<span class="icon-wild" title="Any Skill"></span>', 'WILD')

    text = text.replace(f'<span class="icon-elder_sign" title="Elder Sign"></span>', 'ELDER_SIGN')

    text = text.replace(f'<span class="icon-elder_sign" title="Elder Thing"></span>', 'ELDER_THING')

    text = text.replace(f'<span class="icon-lightning" title="Fast Action"></span>', 'FAST ACTION')

    text = text.replace(f'<span class="icon-auto_fail" title="Auto Fail"></span>', 'TENTACLES')

    return text

def clean_html(text):

    # delete extraneous html
    dirt = [
            '</p>\n</div>',
            '</p>',
            '<p>',
            '<b>',
            '</b>',
            '<br/',
            '<1>',
            '<i>',
            '</i>',
            '><span>',
            '[',
            ']']
    
    for item in dirt:
        
        text = text.replace(item,'')

    return text


def get_cost_xp(results):
    
    cost_xp = str(results.find('div', class_='card-props'))

    cost = re.search(r'Cost:\s+(\d+)', cost_xp).group(1)

    xp = re.search(r'XP:\s+(\d+)', cost_xp).group(1)
    
    return cost, xp

In [8]:
# [
    
#  #title,
#  #sub_title,
#  #faction, 
#  #tipe, 
#  body_slot #### later
#  #traits,
#  cost,
#  #xp,
#  #icons,
#  #ability,
#  #artist,
#  #expansion,
#  #flavor

# ]

In [37]:
results = get_soup('https://arkhamdb.com/card/07305')

In [44]:
str(results.find('div', class_='card-props'))

'<div class="card-props">\n\t\tCost:     4.\n\t\t\tXP: 2.\n\t</div>'

In [87]:
def get_cost_xp(results):
    
    cost_xp = str(results.find('div', class_='card-props'))

    try:
    
        cost = re.search(r'Cost:\s+(\d+)', cost_xp).group(1)
        
    except:
        
        cost = 0

    try:
    
        xp = re.search(r'XP:\s+(\d+)', cost_xp).group(1)
        
    except:
        
        xp = 0
    
    return cost, xp


def get_subtitle(results):
    
    try:
        
        sub_title = results.find('div', class_='card-subname small').text.replace('\n', '').replace('\t', '')
        
    except:
        
        sub_title = ''
        
    return sub_title



def get_traits(results):
    
    try:
        
        traits = results.find('p', class_='card-traits').text.replace('\n', '').replace('\t', '')
        
    except:
        
        traits = ''
        
    return traits

In [88]:
re.search(r'XP:\s+(\d+)', cost_xp).group(1)

'2'

In [89]:
#  #title,
#  #sub_title,
#  #faction, 
#  #tipe, 
#  body_slot #### later
#  #traits,
#  cost,
#  #xp,
#  #icons,
#  #ability,
#  #artist,
#  #expansion,
#  #flavor

In [90]:
def get_event_traits(results):

    title = results.find('a', class_='card-name card-tip').text.replace('\n', '').replace('\t', '')

    sub_title = get_subtitle(results)

    faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

    tipe = results.find('span', class_='card-type').text.replace('\n', '').replace('\t', '')

    traits = get_traits(results)

    cost, xp = get_cost_xp(results)

    icons = get_icons(results)

    ability = get_ability(results, faction)

    artist = results.find('div', class_='card-illustrator').text.replace('\n', '').replace('\t', '')

    flavor = results.find('div', class_='card-flavor small').text.replace('\n', '').replace('\t', '')

    expansion = results.find('div', class_='card-pack').text.replace('\n', '').replace('\t', '').replace('.', '')

    return [title,
             sub_title,
             faction,
             tipe,
             traits,
             cost,
             xp,
             icons,
             ability,
             artist,
             expansion,
             flavor]

In [91]:
event_dict = {'title':[],
             'sub_title':[],
             'faction':[],
             'type':[],
             'traits':[],
             'cost':[],
             'XP':[],
             'icons':[],
             'ability':[],
             'artist':[],
             'expansion':[],
             'flavor':[],
             'url':[]}
    
# for each url get player card info from that page and add each element to skill_traits
for url in events:

    # make html request to arkham db and parse using BS
    results = get_soup(url)

    # get list of card elements card elements
    event_list = get_event_traits(results)

    event_list.append(url)

    print(f'Getting event card {event_list[0]}...')

    # itterate through card element titles and add each to a dictionary
    for i, key in enumerate(event_dict):

        event_dict[key].append(event_list[i])

print("Making dataframe...")

df = pd.DataFrame(event_dict)

Getting investigator card "Eat lead!"...
Getting investigator card "Eat lead!"...
Getting investigator card "Fool me once..."...
Getting investigator card "Get behind me!"...
Getting investigator card "Get over here!"...
Getting investigator card "Get over here!"...
Getting investigator card "Hit me!"...
Getting investigator card "I'll Pay You Back!"...
Getting investigator card "I'll see you in hell!"...
Getting investigator card "I'll take that!"...
Getting investigator card "I'm done runnin'!"...
Getting investigator card "I'm outta here!"...
Getting investigator card "I've got a plan!"...
Getting investigator card "I've got a plan!"...
Getting investigator card "I've had worse…"...
Getting investigator card "I've had worse…"...
Getting investigator card "I've had worse…"...
Getting investigator card "If it bleeds..."...
Getting investigator card "Let God sort them out..."...
Getting investigator card "Let me handle this!"...
Getting investigator card "Look what I found!"...
Getting

Getting investigator card Followed...
Getting investigator card Foresight...
Getting investigator card Forewarned...
Getting investigator card Fortuitous Discovery...
Getting investigator card Fortune or Fate...
Getting investigator card Friends in Low Places...
Getting investigator card Galvanize...
Getting investigator card Gang Up...
Getting investigator card Gaze of Ouraxsh...
Getting investigator card Ghastly Revelation...
Getting investigator card Glimpse the Unthinkable...
Getting investigator card Glimpse the Unthinkable...
Getting investigator card Glory...
Getting investigator card Grievous Wound...
Getting investigator card Grift...
Getting investigator card Grit Your Teeth...
Getting investigator card Guidance...
Getting investigator card Guidance...
Getting investigator card Guided by Faith...
Getting investigator card Hallow...
Getting investigator card Hand of Fate...
Getting investigator card Hand-Eye Coordination...
Getting investigator card Harmony Restored...
Getting

In [94]:
df.sub_title.value_counts()

           238
Yuggoth      1
Yoth         1
Name: sub_title, dtype: int64

In [66]:
title

'.25 Automatic'

In [67]:
sub_title

''

In [68]:
faction

'Rogue'

In [69]:
tipe

'Asset. '

In [70]:
traits

'Item. Weapon. Firearm. Illicit.'

In [71]:
xp

'2'

In [72]:
ability

'Fast. Uses (4 ammo).ACTION Spend 1 ammo: Fight. If the attacked enemy is exhausted, you get +2 COMBAT and deal +1 damage for this attack.REACTION After you evade an enemy at your location: Perform the above Fight ability without spending an action.'

In [73]:
flavor

''

In [74]:
artist

'Mauro Dal Bo'

In [75]:
expansion

'Into the Maelstrom #305'

In [76]:
icons

'COMBAT AGILITY'

In [98]:
for r in range(1,10):
    
    if r == 1:
        
        url = 'https://arkhamdb.com/find?q=t%3Aevent&sort=name&view=list&decks=player&spoilers=hide'

    else:
        
        url = f'https://arkhamdb.com/find?q=t%3Aevent&view=list&sort=name&decks=player&page={r}'

    # create request and soup objects
    try:
        
        html = requests.get(url)
        print('Page found')

    except:
        
        print('Done!!!!!!!!!')
        break

Page found
Page found
Page found
Page found
Page found
Page found
Page found
Page found
Page found


In [ ]:

for r in (10):

    url = f'https://arkhamdb.com/find?q=t%3Aevent&decks=player{num}'

# create request and soup objects
    try:
        
        html = requests.get(url)
        print('Page found')

    except:
        
        print('Done!!!!!!!!!')
        break
        
        
        
soup = BeautifulSoup(html.content, 'html.parser')

# locate urls on page
results = soup.find(id='list')

results = results.find_all('a', class_='card-tip')

# convert urls to string and make a list
results = [str(result['href']) for result in results]

In [68]:
from math import sqrt

def is_prime(num):
    
    
    for r in range (2,int(sqrt(num))+1):
        
        
        if num & r == 0:
            
            print(num,False)
        
        else:
            
            print(num, True) 

In [69]:
is_prime(18)

18 True
18 True
18 False


In [ ]:
def get_key(val):
   
    key_list = list(my_dict.keys())
    val_list = list(my_dict.values())
 

position = val_list.index(100)
print(key_list[position])

In [97]:
# open input file

code_dict = {}
    
with open('coding_qual_input.txt') as file:
    
    for line in file:
        
        num, word = line.split()
        
        code_dict[word] = int(num)


max_num = max(code_dict.values()) 

num_values = 1

num_right = 0



while True:
    
    num_right += num_values
    
    num_values += 1
    
    key_list = list(code_dict.keys())
    val_list = list(code_dict.values())
 
    position = val_list.index(num_right)
    
    print(key_list[position])
    
    if num_right == max_num:
        
        break
        
    elif num_right > max_num:
        
        print("INPUT ERROR")
        break
    
    

down
dont
nine
lot
work
silver
east
duck
done
tone
bit
stop
sun
deal
huge


ValueError: 136 is not in list

In [98]:
code_dict

{'land': 195,
 'sun': 91,
 'too': 266,
 'huge': 120,
 'dont': 3,
 'such': 140,
 'noun': 69,
 'student': 227,
 'brown': 203,
 'complete': 263,
 'play': 174,
 'cook': 162,
 'yard': 40,
 'clock': 253,
 'would': 80,
 'plain': 202,
 'excite': 269,
 'fire': 20,
 'wish': 207,
 'cool': 138,
 'child': 295,
 'past': 128,
 'colony': 29,
 'oil': 51,
 'dog': 34,
 'back': 211,
 'money': 226,
 'kind': 184,
 'open': 238,
 'finger': 101,
 'touch': 283,
 'are': 183,
 'dad': 123,
 'am': 208,
 'modern': 85,
 'meant': 274,
 'ocean': 150,
 'pitch': 167,
 'suit': 18,
 'town': 166,
 'east': 28,
 'over': 189,
 'group': 81,
 'good': 185,
 'down': 1,
 'band': 196,
 'especially': 235,
 'organ': 240,
 'of': 276,
 'out': 79,
 'area': 182,
 'happen': 30,
 'sat': 258,
 'electric': 71,
 'wrote': 163,
 'buy': 65,
 'lot': 10,
 'stop': 78,
 'corn': 297,
 'where': 24,
 'check': 242,
 'live': 239,
 'best': 135,
 'hold': 171,
 'cause': 130,
 'grand': 175,
 'present': 241,
 'indicate': 151,
 'counter': 161,
 'we': 137,
 'lik

In [100]:
list(code_dict.values()).sort()

In [103]:
sorted(code_dict.items(), key=lambda x:x[1])

[('down', 1),
 ('each', 2),
 ('dont', 3),
 ('experiment', 4),
 ('seat', 5),
 ('nine', 6),
 ('prove', 7),
 ('ready', 8),
 ('stead', 9),
 ('lot', 10),
 ('skill', 12),
 ('power', 13),
 ('third', 14),
 ('work', 15),
 ('with', 16),
 ('hole', 17),
 ('suit', 18),
 ('fire', 20),
 ('silver', 21),
 ('young', 22),
 ('where', 24),
 ('heavy', 25),
 ('opposite', 26),
 ('east', 28),
 ('colony', 29),
 ('happen', 30),
 ('chick', 31),
 ('blow', 32),
 ('card', 33),
 ('dog', 34),
 ('straight', 35),
 ('duck', 36),
 ('case', 37),
 ('wild', 38),
 ('cold', 39),
 ('yard', 40),
 ('coast', 42),
 ('cent', 44),
 ('done', 45),
 ('speak', 46),
 ('state', 47),
 ('field', 48),
 ('sister', 49),
 ('planet', 50),
 ('oil', 51),
 ('low', 52),
 ('the', 53),
 ('bird', 54),
 ('tone', 55),
 ('idea', 58),
 ('parent', 60),
 ('blood', 61),
 ('fact', 62),
 ('come', 63),
 ('clothe', 64),
 ('buy', 65),
 ('bit', 66),
 ('trade', 67),
 ('reach', 68),
 ('noun', 69),
 ('oxygen', 70),
 ('electric', 71),
 ('drink', 72),
 ('discuss', 73),
 

In [ ]:
  asset_dict = {'title':[],
                  'sub_title':[],
                  'faction':[],
                  'type':[],
                  'traits':[],
                  'cost':[],
                  'XP':[],
                  'test_icons':[],
                  'ability':[],
                  'health':[],
                  'sanity':[], 
                  'artist':[],
                  'expansion':[],
                  'flavor':[],
                  'url':[]}